<h1> Introduction </h1>

This notebook demonstrates how to scale an implemenation of Full Waveform Inversion using multiple FPGAs with bitstreams containing one or multiple copied compute units.

<h3> Define experiment parameters: </h3>
    
XCLBIN_PATH_DEFAULT => Default path for the .xclbin file containing 1 compute unit  
XCLBIN_PATH_MULTCU => Default path for .xclbin file containing 2 copied compute units  
XRT_ENV_PATH => Path to the Xilinx Runtime setup script.  
DEVICE_NAME_DEFAULT => Default name for the FPGA device  
DIR_PATH => Path to the directory containing the FWI input files.  

In [1]:
XCLBIN_PATH_DEFAULT = "bitstreams/u280_xclbin/500_500_HBM/FullW.xclbin"
XCLBIN_PATH_MULTCU = "bitstreams/u280_xclbin/500_250_HBM/FullW.xclbin"
XRT_ENV_PATH = "/opt/xilinx/xrt/setup.sh"
DEVICE_NAME_DEFAULT="xilinx_u280_xdma_201920_3"

DIR_PATH = "default/"

<h3> Define the worker method </h3> 

Here, we define the Python method which will be executed on each of the Dask workers. This function calls the driver using the data partition it receives, and returns the output data (along with some performance statistics) to the caller (the Dask client). 

We present two methods, the first can be used to execute single compute unit bitstreams, the second demonstrates how we can use multiple dask workers with the same bitstream.

In [2]:
def execute_function(grid_data,kernel,id):
    import numpy as np
    import time
    
    return kernel

    cu = kernel["compute_unit"]
    config = kernel["config"]
    path = kernel["path_to_kernel"]
    start_time = time.time()
    grid_data = grid_data
    
    from pynq import Overlay, allocate, Device, lib
    from FWIDriver import FWI

    resolution = config["Freq"]["nTotal"] * config["nSources"] * config["nReceivers"]
    gridsize = len(grid_data)   
    config["tolerance"] = 9.99*10**-7
    config["max"] = 1000
    
    import socket
    st = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    try:       
        st.connect(('10.255.255.255', 1))
        IP = st.getsockname()[0]
    except Exception:
        IP = '127.0.0.1'
    finally:
        st.close()

    # Load the overlay
    devices = Device.devices
    ol = Overlay(path, download=False, device=devices[0])
    
    return IP, cu

    # Allocate the buffers
    try:
        A = allocate(shape=(resolution,gridsize), dtype=np.complex64, target=getattr(ol,kernel["functions"][0]["dotprod_"+str(cu)][0]))
    except Exception as e:
        return f" e:{e} IP: {IP}"
    
    B = allocate(shape=(gridsize,), dtype=np.float32, target=getattr(ol,kernel["functions"][0]["dotprod_"+str(cu)][1]))
    C = allocate(shape=(resolution,), dtype=np.complex64, target=getattr(ol,kernel["functions"][0]["dotprod_"+str(cu)][2]))

    D = allocate(shape=(resolution,gridsize), dtype=np.complex64,  target=getattr(ol,kernel["functions"][1]["update_"+str(cu)][0]))
    E = allocate(shape=(resolution,),dtype=np.complex64,  target=getattr(ol,kernel["functions"][1]["update_"+str(cu)][1]))
    F = allocate(shape=(gridsize), dtype=np.complex64, target=getattr(ol,kernel["functions"][1]["update_"+str(cu)][2]))

    # set up the kernel IP's
    dotprod = getattr(ol,"dotprod_"+str(cu))
    update = getattr(ol,"update_"+str(cu))


    fwi = FWI(A,B,C,D,E,F,dotprod,update,config,resolution,gridsize,True)

    # pre process the grid data
    fwi.pre_process(grid_data)

    # reconstruct the grid by performing Full Wavefrom Inversion
    chi = fwi.reconstruct()

    total_time = time.time() - start_time

    # free all the buffers
    A.freebuffer()
    B.freebuffer()
    C.freebuffer()
    D.freebuffer()
    E.freebuffer()
    F.freebuffer()
    ol.free()
    
    dict_t = {
    "id": id,
    "cu": cu,
    "time": total_time,
    "chi":chi,
    "IP":IP
    }
    return dict_t
    


<h2> Setup method for multiple compute units </h2>

In [3]:
def setup_multcu():
    from pynq import Device, Overlay
    ol = Overlay(XCLBIN_PATH_MULTCU,download=True,device=Device.devices[0])
    return 'setup succesful'
                        

Second method!

In [4]:
# def run_on_worker(grid_data,kernel,index):
#     from dask import delayed, compute
#     from pynq import Overlay, allocate, Device, lib
#     from multiprocessing import Queue, Process

#     devices = Device.devices
#     ol = Overlay(kernel["path_to_kernel"], download=True, device=devices[0])
    
#     def execute_function(queue, grid_data,kernel,path,index,config):
#         import numpy as np
#         import time
        
#         start_time = time.time()


#         from pynq import Overlay, allocate, Device, lib
#         from FWIDriver import FWI
        
        
#         resolution = config["Freq"]["nTotal"] * config["nSources"] * config["nReceivers"]
#         gridsize = len(grid_data)   
#         config["tolerance"] = 9.99*10**-7
#         config["max"] = 1000

#         # Load the overlay
#         devices = Device.devices
#         ol = Overlay(path, download=True, device=devices[0])
        
#         # Allocate the buffers
#         A = allocate(shape=(resolution,gridsize), dtype=np.complex64, target=getattr(ol,kernel["functions"]["dotprod_"+index][0]))
#         B = allocate(shape=(gridsize,), dtype=np.float32, target=getattr(ol,kernel[index]["dotprod_"+index][1]))
#         C = allocate(shape=(resolution,), dtype=np.complex64, target=getattr(ol,kernel[index]["dotprod_"+index][2]))

#         D = allocate(shape=(resolution,gridsize), dtype=np.complex64,  target=getattr(ol,kernel[index]["update_"+index][0]))
#         E = allocate(shape=(resolution,),dtype=np.complex64,  target=getattr(ol,kernel[index]["update_"+index][1]))
#         F = allocate(shape=(gridsize), dtype=np.complex64, target=getattr(ol,kernel[index]["update_"+index][2]))

#         # set up the kernel IP's
#         dotprod = getattr(ol,"dotprod_"+index)
#         update = getattr(ol,"update_"+index)
        
        
#         fwi = FWI(A,B,C,D,E,F,dotprod,update,config,resolution,gridsize,True)
        
#         # pre process the grid data
#         fwi.pre_process(grid_data)
        
#         # reconstruct the grid by performing Full Wavefrom Inversion
#         chi = fwi.reconstruct()
        
#         total_time = time.time() - start_time

#         # free all the buffers
#         A.freebuffer()
#         B.freebuffer()
#         C.freebuffer()
#         D.freebuffer()
#         E.freebuffer()
#         F.freebuffer()
        
#         dict_t = {
#         "index": index,
#         "time": total_time,
#         "chi":chi
#         }
#         queue.put(dict_t)
    

#     # Create a subprocess to handle multiple compute units, if we only use single compute units bitstreams we can map the execute functions directly
#     # TODO: example of that?
#     if kernel["no_instances"]==1:
#         q = Queue()
#         cu = 0
#         p = Process(target=execute_function,args=(q, grid_data[cu],kernel["functions"][cu],kernel["path_to_kernel"],str(cu+1),kernel["config"]))
#         p.start()
#         result = q.get()
#         p.join()
#     else:
#         p = []
#         q = []
#         result = []
#         for cu in range(kernel["no_instances"]):
#             q.append(Queue())
#             p.append(Process(target=execute_function,args=(q[cu], grid_data[cu],kernel["functions"][cu],kernel["path_to_kernel"],str(cu+1),kernel["config"])))
#             p[cu].start()
#         for cu in range(kernel["no_instances"]):
#             result.append(q[cu].get())
#         for cu in range(kernel["no_instances"]):
#             p[cu].join()
     
#     # Free the overlay and return the results
#     ol.free()
#     return result
            


In [5]:
from Octoray import Octoray

# Create an octoray instance with the 
# octoray = Octoray(ssh_cluster=True, scheduler="10.1.212.126",scheduler_port=8786, hosts=["10.1.212.127","10.1.212.126"], config_file="cluster_config.json")
octoray = Octoray(ssh_cluster=False,config_file="cluster_config.json")


# first load in the data
import json
import copy
import time


# Load in data and config settings
data = []
with open(DIR_PATH+"input/"+"10x10_100"+".txt") as f:
    for l in f:
        data.append(float(l))
        
data.extend(data)

config = None
with open(DIR_PATH+"input/GenericInput.json") as f:
    config = json.load(f)

#set specific configurations for different types of kernels
single_cu_config = config
double_cu_config = copy.deepcopy(config)
single_cu_config["ngrid"]["x"]=50
double_cu_config["ngrid"]["x"]=25

# Configure the kernels by specifying the path to the bitstream, number of compute units, batchsize per compute unit and the function names and variables with their respective memory banks.
single_cu = octoray.create_kernel(XCLBIN_PATH_DEFAULT,1,500,[[{"dotprod_1":["HBM0","HBM1","HBM2"]},{"update_1":["HBM3","HBM4","HBM5"]}]],single_cu_config)

double_cu = octoray.create_kernel(XCLBIN_PATH_MULTCU,2,250,[[{"dotprod_1":["HBM0","HBM1","HBM2"]},{"update_1":["HBM6","HBM7","HBM8"]}],
                                                [{"dotprod_2":["HBM3","HBM4","HBM5"]},{"update_2":["HBM9","HBM10","HBM11"]}]],double_cu_config)

# Finally, add the kernels you want to execute
# data_split, kernels_split = octoray.setup_cluster(data,single_cu,copy.deepcopy(single_cu))
data_split, kernels_split = octoray.setup_cluster(data,double_cu,copy.deepcopy(double_cu))


Initializing OctoRay with client ip: 10.1.212.129
[{'nthreads': 1, 'n_workers': 2, 'preload': 'pynqimport.py', 'nanny': '0', 'memory_limit': 0}, {'nthreads': 1, 'n_workers': 2, 'preload': 'pynqimport.py', 'nanny': '0', 'memory_limit': 0}]
2
['10.1.212.129', '10.1.212.129', '10.1.212.127']


distributed.deploy.ssh - INFO - distributed.scheduler - INFO - Clear task state
distributed.deploy.ssh - INFO - distributed.scheduler - INFO -   Scheduler at:   tcp://10.1.212.129:8786
distributed.deploy.ssh - INFO - distributed.utils - INFO - Reload module pynqimport from .py file
distributed.deploy.ssh - INFO - distributed.utils - INFO - Reload module pynqimport from .py file
distributed.deploy.ssh - INFO - distributed.preloading - INFO - Import preload module: pynqimport.py
distributed.deploy.ssh - INFO - distributed.utils - INFO - Reload module pynqimport from .py file
distributed.deploy.ssh - INFO - distributed.preloading - INFO - Import preload module: pynqimport.py
distributed.deploy.ssh - INFO - distributed.preloading - INFO - Import preload module: pynqimport.py
distributed.deploy.ssh - INFO - distributed.utils - INFO - Reload module pynqimport from .py file
distributed.deploy.ssh - INFO - distributed.worker - INFO -       Start worker at:   tcp://10.1.212.129:39425
distribute

Waiting until workers are set up on remote machines...
Current amount of workers: 4


In [ ]:
# Execute the added kernels on the workers.
# print(kernels_split)

# print(data_split)
results = octoray.execute(execute_function,data_split,kernels_split,range(len(octoray.kernels)))

print(results)

In [23]:

# print(len(data_split[1]))

def x(*args):
    distributed_args = []
#     for i,arg in enumerate(args):
# args[kernel][hosts][worker]
    
    for i, item in enumerate(self.hosts):
        if isinstance(item,dict):
            futures.append(self.client.submit(func,*kernel_args[0],i,workers=item["host"]))
        elif isinstance(item,list):
            for x, cu in enumerate(item):
                kernel_args = []
                kernel_args.append(args[0])
                args[0][0]
                futures.append(self.client.submit(func,args[i],i,workers=cu["host"]))
            
            
    for arg in args:
        distributed
    
    
    print(t[0])
    print(t[1])
#     print(args[0][0][0])
#     print(args[1][0][0])
#     print("(----)")
            
x(data_split,kernels_split)
# result = octoray.execute_hybrid(setup_multcu,execute_function,data_split,kernels_split)

# print(result)

TypeError: x() takes 1 positional argument but 2 were given

<h3> Graceful shutdown for OpenSSH version >= 7.9 </h3>

In [ ]:
octoray.shutdown()

<h3> Ungraceful shutdown for OpenSSH version < 7.9 </h3>

In [ ]:
await octoray.fshutdown()

In [20]:
from dask.distributed import Client, progress

client = Client("tcp://10.1.212.126:8786")

# print(len(octoray.kernels))
# print(len(client.scheduler_info()["workers"]))
client

<Client: 'tcp://10.1.212.126:8786' processes=2 threads=2>